In [0]:
def upsert_silver(microbatch_df, batch_id):
    microbatch_df.createOrReplaceTempView("microbatch")

    spark.sql("""
    MERGE INTO retail_lakehouse.silver.sales_silver t
    USING (
        SELECT *
        FROM (
            SELECT *,
                   ROW_NUMBER() OVER (
                     PARTITION BY order_id
                     ORDER BY ingest_date DESC, ingestion_ts DESC
                   ) rn
            FROM microbatch
            WHERE order_id IS NOT NULL
        )
        WHERE rn = 1
    ) s
    ON t.order_id = s.order_id
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
    """)

(
    spark.readStream
         .table("retail_lakehouse.bronze.sales_bronze")
         .writeStream
         .foreachBatch(upsert_silver)
         .option("checkpointLocation",
                 "/Volumes/retail_lakehouse/bronze/checkpoints/silver")
         .start()
)
